<a href="https://colab.research.google.com/github/priyanshgupta1998/Natural-language-processing-NLP-/blob/master/Black_offer_Assignment/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [120]:
path = '/content/drive/My Drive/Data Science'
import os
import pandas as pd
print(os.listdir(path))

['Output Data Structure.xlsx', 'constraining_dictionary.xlsx', 'Text Analysis.docx', 'cik_list.xlsx', 'Objective.docx', 'uncertainty_dictionary.xlsx']


In [149]:
file1 = path + '/cik_list.xlsx'
cik = pd.read_excel(file1)
print(cik.shape)
cik.head()

(152, 6)


,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [0]:
cik['SECFNAME_link'] = cik['SECFNAME'].apply(lambda x:'https://www.sec.gov/Archives/' + str(x) ) 

In [151]:
cik.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,SECFNAME_link
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,https://www.sec.gov/Archives/edgar/data/3662/0...
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,https://www.sec.gov/Archives/edgar/data/3662/0...
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,https://www.sec.gov/Archives/edgar/data/3662/0...
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,https://www.sec.gov/Archives/edgar/data/3662/0...
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,https://www.sec.gov/Archives/edgar/data/3662/0...


In [152]:
for i in range(5):
  print(list(cik['SECFNAME_link'])[i])

https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt
https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-001001.txt
https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt
https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002145.txt
https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-001203.txt




---



---



---



---



In [127]:
#making the stopword set from basic english and the given list of stopwords'
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stopset = set(w.upper() for w in stopwords.words('english'))
#adding more stopwords from text file of stopwords
import glob
path = "StopWords*.txt"
glob.glob(path)
for filename in glob.glob(path):
    with open(filename, 'r') as f:
        text = f.read()
        text = re.sub(r"\s+\|\s+[\w]*" , "", text)        
        stopset.update(text.upper().split())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [128]:
# syllables count (will be used in complex word count)
from nltk.corpus import cmudict
nltk.download('cmudict')
d = cmudict.dict()
def sylla_num(word):
    try:
      #if word is found in cmudict
      return max([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])
    except KeyError:  
      #claculate syllables
      count = 0
      vowels = 'aeiou'
      word = word.lower()
      if word[0] in vowels:
          count +=1
      for index in range(1,len(word)):
          if word[index] in vowels and word[index-1] not in vowels:
              count +=1
      if word.endswith('e'):
          count -= 1
      if word.endswith('le'):
          count+=1
      if count == 0:
          count +=1
      return count      

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [129]:
# section names
MDA = "Management's Discussion and Analysis"
QQDMR = "Quantitative and Qualitative Disclosures about Market Risk"
RF = "Risk Factors"
section_name = ['MDA','QQDMR',"RF"]
section = [MDA.upper(),QQDMR.upper(),RF.upper()]
variables = ['positive_score','negative_score','polarity_score','average_sentence_length', 'percentage_of_complex_words',\
                   'fog_index','complex_word_count','word_count','uncertainty_score','constraining_score', 'positive_word_proportion',\
                   'negative_word_proportion', 'uncertainty_word_proportion', 'constraining_word_proportion' ]
import itertools
df_col = [sec.lower() + '_' + var for sec,var in itertools.product(section_name,variables) ]
print(df_col)

Series([], Name: constraining_words_whole_report, dtype: float64)
['mda_positive_score', 'mda_negative_score', 'mda_polarity_score', 'mda_average_sentence_length', 'mda_percentage_of_complex_words', 'mda_fog_index', 'mda_complex_word_count', 'mda_word_count', 'mda_uncertainty_score', 'mda_constraining_score', 'mda_positive_word_proportion', 'mda_negative_word_proportion', 'mda_uncertainty_word_proportion', 'mda_constraining_word_proportion', 'qqdmr_positive_score', 'qqdmr_negative_score', 'qqdmr_polarity_score', 'qqdmr_average_sentence_length', 'qqdmr_percentage_of_complex_words', 'qqdmr_fog_index', 'qqdmr_complex_word_count', 'qqdmr_word_count', 'qqdmr_uncertainty_score', 'qqdmr_constraining_score', 'qqdmr_positive_word_proportion', 'qqdmr_negative_word_proportion', 'qqdmr_uncertainty_word_proportion', 'qqdmr_constraining_word_proportion', 'rf_positive_score', 'rf_negative_score', 'rf_polarity_score', 'rf_average_sentence_length', 'rf_percentage_of_complex_words', 'rf_fog_index', 'rf_

In [131]:
path = '/content/drive/My Drive/Data Science'
file2 = path + '/uncertainty_dictionary.xlsx'
uncertain = pd.read_excel(file2)
print(uncertain.shape)
uncertain.head()

(297, 1)


,Word
0,ABEYANCE
1,ABEYANCES
2,ALMOST
3,ALTERATION
4,ALTERATIONS


In [132]:
path = '/content/drive/My Drive/Data Science'
file4 = path + '/constraining_dictionary.xlsx'
constrain = pd.read_excel(file4)
print(constrain.shape)
constrain.head()

(184, 1)


,Word
0,ABIDE
1,ABIDING
2,BOUND
3,BOUNDED
4,COMMIT


In [133]:
total_word = pd.concat([uncertain,constrain] , ignore_index=True )
print(total_word.shape)

(481, 1)


In [0]:
uncertainty_dict = list(uncertain['Word'])
constraining_dict = list(constrain['Word'])

In [0]:
from urllib.request import urlopen as ureq
from bs4 import BeautifulSoup
import numpy as np
import requests
import re
from nltk import sent_tokenize
from textblob import TextBlob

df = pd.DataFrame(columns=df_col)
print(df.shape)

for i in range(len(cik)):
  url =  list(cik['SECFNAME_link'])[i]
  page = requests.get(url)
  html = BeautifulSoup(page.content,'html.parser')
  df.loc[i] = np.zeros(len(df.columns))  
  
  for j in range(3):
    if i in [63,64]:
      continue
    exp = r".*(?P<start>ITEM [\d]\. " + re.escape(section[j]) + r")(?P<MDA>.*)(?P<body>[\s\S]*)(?P<end>ITEM \d|SIGNATURES)"
    regexp = re.compile(exp)
    try:
      print(i)
      s = regexp.search(str(html))
    except:
      pass
    # print(type(s))

    if (s):
      data = s.group('body')
      soup = BeautifulSoup(data, "html.parser")
      text = soup.get_text()  # extract only text data
      text = re.sub('\[[^]]*\]', '', text)  #remove squares
      text = re.sub('[\d%/$]', '', text)  # remove all the numbers
      sent_list = sent_tokenize(text)  #tokenizatopn the string into sentences
      sentence_length = len(sent_list)

      sample = text.split()   #splited into words

      rem_Nascii = []  #remove non ascii
      for word in sample: 
          new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
          rem_Nascii.append(new_word)
      words = rem_Nascii
      conv_upr = []   #convert into upper character
      for word in words:
          new_word = word.upper()
          conv_upr.append(new_word)
      words = conv_upr
      rm_punc = []  #remove punctuation
      for word in words:
          new_word = re.sub(r'[^\w\s]', '', word)
          if new_word != '':
              rm_punc.append(new_word)
      words = rm_punc  
      rm_stopwod = []  #remove stopwords
      for word in words:
          if word not in stopset:
              rm_stopwod.append(word)
      sample = rm_stopwod #finally data clearning done
      word_count = len(sample)

      complex_word_count = 0
      for word in sample:
          if sylla_num(word.lower()) > 3: #call sylla_num function
              complex_word_count += 1

      #calculate the fog index 
      average_sentence_length = word_count/sentence_length
      percentage_of_complex_words = complex_word_count/word_count
      fog_index = 0.4 * (average_sentence_length + percentage_of_complex_words)

      #calculate the other variables
      positive_score = 0
      negative_score = 0
      uncertainty_score = 0
      constraining_score = 0
      for word in sample:
          if word in list(total_word.Word):
              testimonial = TextBlob(word)
              if testimonial.sentiment.polarity> 0:   #polarity checking 
                  positive_score += 1
              if testimonial.sentiment.polarity< 0:
                  negative_score += 1
              if word in uncertainty_dict:
                  uncertainty_score += 1
              if word in constraining_dict:
                  constraining_score += 1
      #print(positive_score)
      polarity_score = (positive_score-negative_score)/(positive_score + negative_score + .000001)
      positive_word_proportion = positive_score/word_count
      negative_word_proportion = negative_score/word_count
      uncertainty_word_proportion = uncertainty_score/word_count
      constraining_word_proportion = constraining_score/word_count

      df.loc[i][section_name[j].lower() + "_positive_score"] = positive_score
      df.loc[i][section_name[j].lower() + "_negative_score"] = negative_score
      df.loc[i][section_name[j].lower() + "_polarity_score"] = polarity_score
      df.loc[i][section_name[j].lower() + "_average_sentence_length"] = average_sentence_length
      df.loc[i][section_name[j].lower() + "_percentage_of_complex_words"] = percentage_of_complex_words
      df.loc[i][section_name[j].lower() + "_fog_index"] = fog_index
      df.loc[i][section_name[j].lower() + "_complex_word_count"] = complex_word_count
      df.loc[i][section_name[j].lower() + "_word_count"] = word_count
      df.loc[i][section_name[j].lower() + "_uncertainty_score"] = uncertainty_score
      df.loc[i][section_name[j].lower() + "_constraining_score"] = constraining_score
      df.loc[i][section_name[j].lower() + "_positive_word_proportion"] = positive_word_proportion
      df.loc[i][section_name[j].lower() + "_negative_word_proportion"] = negative_word_proportion
      df.loc[i][section_name[j].lower() + "_uncertainty_word_proportion"] = uncertainty_word_proportion
      df.loc[i][section_name[j].lower() + "_constraining_word_proportion"] = constraining_word_proportion  

In [141]:
print(df.shape)
df.head()

(152, 42)


,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainty_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_uncertainty_word_proportion,mda_constraining_word_proportion,qqdmr_positive_score,qqdmr_negative_score,qqdmr_polarity_score,qqdmr_average_sentence_length,qqdmr_percentage_of_complex_words,qqdmr_fog_index,qqdmr_complex_word_count,qqdmr_word_count,qqdmr_uncertainty_score,qqdmr_constraining_score,qqdmr_positive_word_proportion,qqdmr_negative_word_proportion,qqdmr_uncertainty_word_proportion,qqdmr_constraining_word_proportion,rf_positive_score,rf_negative_score,rf_polarity_score,rf_average_sentence_length,rf_percentage_of_complex_words,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion
0,0.0,12.0,-1.0,11.497908,0.184134,4.672817,506.0,2748.0,32.0,15.0,0.0,0.004367,0.011645,0.005459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,15.0,-1.0,15.200000,0.166842,6.146737,317.0,1900.0,71.0,3.0,0.0,0.007895,0.037368,0.001579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,32.0,-1.0,12.096203,0.175596,4.908720,839.0,4778.0,94.0,53.0,0.0,0.006697,0.019674,0.011093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [146]:
whole_report = pd.Series(name='constraining_words_whole_report')
print(whole_report)
for i in range(len(cik)):
  url =  list(cik['SECFNAME_link'])[i]
  page = requests.get(url)
  try:
    print(i , end=' ')
    text = str(BeautifulSoup(page.content,'html.parser'))
    #constraining_words_whole_report
    whole_report.loc[i] = 0
    count = 0
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()  # extract only text data
    text = re.sub('\[[^]]*\]', '', text)  #remove squares
    text = re.sub('[\d%/$]', '', text)  # remove all the numbers
    oug = text.split()
    for word in oug:
        if word in constraining_dict:
            count += 1
    whole_report.loc[i] = count
  except:
    whole_report.loc[i]= 0

Series([], Name: constraining_words_whole_report, dtype: float64)
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 

In [147]:
print(whole_report.shape)
whole_report.head()

(152,)


0    55
1    41
2     0
3    38
4     0
Name: constraining_words_whole_report, dtype: int64

In [155]:
# joing the files for output formate
result = pd.concat([cik,df,whole_report], axis = 1)
print(result.shape)
result.head()

(152, 58)


,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,SECFNAME_link,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,SECFNAME_link,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainty_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_uncertainty_word_proportion,mda_constraining_word_proportion,qqdmr_positive_score,qqdmr_negative_score,qqdmr_polarity_score,qqdmr_average_sentence_length,qqdmr_percentage_of_complex_words,qqdmr_fog_index,qqdmr_complex_word_count,qqdmr_word_count,qqdmr_uncertainty_score,qqdmr_constraining_score,qqdmr_positive_word_proportion,qqdmr_negative_word_proportion,qqdmr_uncertainty_word_proportion,qqdmr_constraining_word_proportion,rf_positive_score,rf_negative_score,rf_polarity_score,rf_average_sentence_length,rf_percentage_of_complex_words,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,12.0,-1.0,11.497908,0.184134,4.672817,506.0,2748.0,32.0,15.0,0.0,0.004367,0.011645,0.005459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55,55
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,15.0,-1.0,15.200000,0.166842,6.146737,317.0,1900.0,71.0,3.0,0.0,0.007895,0.037368,0.001579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41,41
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,32.0,-1.0,12.096203,0.175596,4.908720,839.0,4778.0,94.0,53.0,0.0,0.006697,0.019674,0.011093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38,38
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [137]:
file3 = path + '/Output Data Structure.xlsx'
output = pd.read_excel(file3 )
print(output.shape)
print(output.columns)
output.head()

(0, 49)


,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainty_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_uncertainty_word_proportion,mda_constraining_word_proportion,qqdmr_positive_score,qqdmr_negative_score,qqdmr_polarity_score,qqdmr_average_sentence_length,qqdmr_percentage_of_complex_words,qqdmr_fog_index,qqdmr_complex_word_count,qqdmr_word_count,qqdmr_uncertainty_score,qqdmr_constraining_score,qqdmr_positive_word_proportion,qqdmr_negative_word_proportion,qqdmr_uncertainty_word_proportion,qqdmr_constraining_word_proportion,rf_positive_score,rf_negative_score,rf_polarity_score,rf_average_sentence_length,rf_percentage_of_complex_words,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report


In [0]:
result.to_csv('Output _Data_Structures.csv' , index =False)

In [0]:
result.to_excel('Output_Data_Structures.xlsx')